This notebook will focus on reading the .csv data from our dataset, inspecting the data

In [7]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path("../data/")
PICKLE_PATH = Path("../pickled_data/")

### Play Data
This set stores information about individual plays or events within a game. There are different event types corresponding to the moments of statistical interest within a hockey game. Some event types are associated with x, y coordinates giving location on the ice.

In [8]:
game_plays = pd.read_csv(DATA_PATH / "game_plays.csv")
game_plays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050529 entries, 0 to 5050528
Data columns (total 18 columns):
 #   Column               Dtype  
---  ------               -----  
 0   play_id              object 
 1   game_id              int64  
 2   team_id_for          float64
 3   team_id_against      float64
 4   event                object 
 5   secondaryType        object 
 6   x                    float64
 7   y                    float64
 8   period               int64  
 9   periodType           object 
 10  periodTime           int64  
 11  periodTimeRemaining  float64
 12  dateTime             object 
 13  goals_away           int64  
 14  goals_home           int64  
 15  description          object 
 16  st_x                 float64
 17  st_y                 float64
dtypes: float64(7), int64(5), object(6)
memory usage: 693.6+ MB


In [44]:
null_plays = game_plays.isna().sum()
null_plays

play_id                      0
game_id                      0
team_id_for             932705
team_id_against         932705
event                        0
secondaryType          3868513
x                      1134364
y                      1134333
period                       0
periodType                   0
periodTime                   0
periodTimeRemaining     193019
dateTime                     0
goals_away                   0
goals_home                   0
description                  0
st_x                   1134364
st_y                   1134364
dtype: int64

There are some significant missing data at a glance, however we know that this by design. Certain events are missing team info because they are "neutral" and don't pertain to a particular team. X, Y coords are not recorded for every event type, and secondaryType is only recorded for shot events.

In [45]:
game_plays['event'].value_counts()

event
Faceoff                     894041
Shot                        845164
Hit                         706096
Stoppage                    698342
Blocked Shot                431922
Missed Shot                 357214
Giveaway                    273287
Penalty                     247828
Takeaway                    211881
Goal                        148992
Period Start                 50140
Period Ready                 50073
Period Official              50067
Period End                   50067
Game End                     15033
Game Scheduled               15027
Game Official                 2351
Official Challenge            1470
Shootout Complete             1456
Early Intermission Start        35
Early Intermission End          35
Emergency Goaltender             8
Name: count, dtype: int64

Example of an event type without certain data by design:

In [46]:
game_plays[game_plays['event'] == 'Shootout Complete']

,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,dateTime,goals_away,goals_home,description,st_x,st_y
3202,2015020346_404,2015020346,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,300.0,2015-11-29 03:00:11,2,2,End of Shootout,NaN,NaN
5392,2016021020_331,2016021020,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,300.0,2017-03-14 03:51:56,3,3,End of Shootout,NaN,NaN
8176,2015020086_320,2015020086,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,300.0,2015-10-21 03:51:04,4,4,End of Shootout,NaN,NaN
13662,2017020930_355,2017020930,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,300.0,2018-02-23 02:47:48,3,3,End of Shootout,NaN,NaN
19595,2016020674_323,2016020674,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,300.0,2017-01-19 03:58:58,5,5,End of Shootout,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4977055,2018021254_377,2018021254,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,0.0,2019-04-06 02:51:27,2,2,End of Shootout,NaN,NaN
4979264,2018021258_297,2018021258,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,0.0,2019-04-06 23:45:45,2,2,End of Shootout,NaN,NaN
4979677,2018021259_409,2018021259,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,0.0,2019-04-07 03:09:43,5,5,End of Shootout,NaN,NaN
4983998,2018021258_297,2018021258,NaN,NaN,Shootout Complete,NaN,NaN,NaN,5,SHOOTOUT,0,0.0,2019-04-06 23:45:45,2,2,End of Shootout,NaN,NaN


In [47]:
duplicate_plays = game_plays.groupby(['play_id'])['game_id'].count() > 1
duplicate_plays.value_counts()

game_id
False    3383597
True      833466
Name: count, dtype: int64

There is some duplication based on the play_id. We should attempt to drop duplicates with the same play_id, in the same game, happening at the same exact time.

In [48]:
subset_columns = ['play_id', 'game_id', 'dateTime']
game_plays = game_plays.drop_duplicates(subset=subset_columns, keep="first")

In [49]:
duplicate_plays = game_plays.groupby(subset_columns)['game_id'].count() > 1
duplicate_plays.value_counts()

game_id
False    4217063
Name: count, dtype: int64

With no more duplicates, we can take a look at the data we're most interested in, shots and goals

In [50]:
shots = game_plays[game_plays['event'] == "Shot"]
shots.head()

,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,dateTime,goals_away,goals_home,description,st_x,st_y
4,2016020045_5,2016020045,16.0,4.0,Shot,Wrist Shot,-71.0,9.0,1,REGULAR,54,1146.0,2016-10-19 01:41:44,0,0,Artem Anisimov Wrist Shot saved by Michal Neuv...,71.0,-9.0
7,2016020045_8,2016020045,4.0,16.0,Shot,Wrist Shot,56.0,-7.0,1,REGULAR,69,1131.0,2016-10-19 01:42:41,0,1,Dale Weise Wrist Shot saved by Corey Crawford,56.0,-7.0
11,2016020045_12,2016020045,16.0,4.0,Shot,Slap Shot,-37.0,-24.0,1,REGULAR,133,1067.0,2016-10-19 01:43:45,0,1,Duncan Keith Slap Shot saved by Michal Neuvirth,37.0,24.0
16,2016020045_17,2016020045,4.0,16.0,Shot,Wrist Shot,57.0,-20.0,1,REGULAR,168,1032.0,2016-10-19 01:44:54,0,1,Wayne Simmonds Wrist Shot saved by Corey Crawford,57.0,-20.0
18,2016020045_19,2016020045,4.0,16.0,Shot,Slap Shot,34.0,14.0,1,REGULAR,171,1029.0,2016-10-19 01:45:01,0,1,Shayne Gostisbehere Slap Shot saved by Corey C...,34.0,14.0


In [51]:
shots['secondaryType'].value_counts()

secondaryType
Wrist Shot     362395
Slap Shot      131801
Snap Shot       99914
Backhand        56995
Tip-In          29572
Deflected        9418
Wrap-around      8246
Name: count, dtype: int64

In [52]:
shots['secondaryType'].isna().sum()

np.int64(24)

There are very few missing shots, we can fill them with the mode 'Wrist Shot'

In [68]:
shot_mask = game_plays['event'] == 'Shot'
game_plays.loc[shot_mask, 'secondaryType'] = game_plays.loc[shot_mask, 'secondaryType'].fillna('Wrist Shot')

In [71]:
game_plays[shot_mask]['secondaryType'].isna().sum()

np.int64(0)

With the missing values for secondaryType populated, this data is ready to go!

In [72]:
game_plays.to_pickle(PICKLE_PATH / "game_plays")

### Game Data
This table gives data about the game itself. Each game has an ID, the season it was played in, the teams that participated, the score and outcome, and venue data

In [5]:
games = pd.read_csv(DATA_PATH / "game.csv")
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26305 entries, 0 to 26304
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   game_id                 26305 non-null  int64 
 1   season                  26305 non-null  int64 
 2   type                    26305 non-null  object
 3   date_time_GMT           26305 non-null  object
 4   away_team_id            26305 non-null  int64 
 5   home_team_id            26305 non-null  int64 
 6   away_goals              26305 non-null  int64 
 7   home_goals              26305 non-null  int64 
 8   outcome                 26305 non-null  object
 9   home_rink_side_start    25109 non-null  object
 10  venue                   26305 non-null  object
 11  venue_link              26305 non-null  object
 12  venue_time_zone_id      26305 non-null  object
 13  venue_time_zone_offset  26305 non-null  int64 
 14  venue_time_zone_tz      26305 non-null  object
dtypes:

In [6]:
games.shape

(26305, 15)

In [13]:
games['season'].unique()

array([20162017, 20172018, 20152016, 20142015, 20132014, 20122013,
       20112012, 20102011, 20092010, 20082009, 20072008, 20062007,
       20022003, 20032004, 20052006, 20002001, 20012002, 20192020,
       20182019])

We have no null data in the games set, let's check for duplicates.

In [5]:
duplicate_games = games.groupby(['game_id'])['season'].count() > 1
duplicate_games.value_counts()

season
False    21165
True      2570
Name: count, dtype: int64

Looks like there is some duplication. We can say for sure that no two games should have the same id, season, venue, and teams involved. If all those factors were the same, the game row is certainly duplicate

In [6]:
subset_columns = ['game_id', 'season', 'venue', 'away_team_id', 'home_team_id']
games = games.drop_duplicates(subset=subset_columns, keep="first")

In [7]:
games.shape

(23735, 15)

In [8]:
games.to_pickle(PICKLE_PATH+"games")

### Game Teams stats
This table gives aggregate statistics and other team data for each game. Goals, shots, hits, penalties, faceoff win %, etc.

In [15]:
game_teams = pd.read_csv(DATA_PATH / "game_teams_stats.csv")
game_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52610 entries, 0 to 52609
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 52610 non-null  int64  
 1   team_id                 52610 non-null  int64  
 2   HoA                     52610 non-null  object 
 3   won                     52610 non-null  bool   
 4   settled_in              52610 non-null  object 
 5   head_coach              52582 non-null  object 
 6   goals                   52602 non-null  float64
 7   shots                   52602 non-null  float64
 8   hits                    47682 non-null  float64
 9   pim                     52602 non-null  float64
 10  powerPlayOpportunities  52602 non-null  float64
 11  powerPlayGoals          52602 non-null  float64
 12  faceOffWinPercentage    30462 non-null  float64
 13  giveaways               47682 non-null  float64
 14  takeaways               47682 non-null

In [16]:
game_teams.shape

(52610, 17)

There is some null data in faceoff win %, giveaways, takeaways, hits. Hits were not always a tracked stat and were only begun to be measured at some point during the timeline of this data, so we are expecting some missing values. For the others, the data isn't in areas we anticipate investigating heavily, so we will forego imputing any values for now.

Let's move on to inspecting for duplication.

In [17]:
team_duplicates = game_teams.groupby(['game_id', 'team_id'])['won'].count() > 1
team_duplicates.value_counts()

won
False    42330
True      5140
Name: count, dtype: int64

No game should have the same id with multiple entries for two teams of the same id. Ex. for each game there should be two entries, one for each team.

In [13]:
subset_columns = ['game_id', 'team_id']
game_teams = game_teams.drop_duplicates(subset=subset_columns, keep="first")

In [14]:
game_teams.shape

(47470, 17)

Duplicates dropped, and a consistent result mathematically with our games set. 

In [18]:
game_teams.to_pickle(PICKLE_PATH / "game_teams")

### Game Plays Players
This table is an intermediate mapping table for matching game_plays to player_info. It contains a play_id, game_id, and player_id along with playerType

In [20]:
game_plays_players = pd.read_csv(DATA_PATH / "game_plays_players.csv")
game_plays_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7586604 entries, 0 to 7586603
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   play_id     object
 1   game_id     int64 
 2   player_id   int64 
 3   playerType  object
dtypes: int64(2), object(2)
memory usage: 231.5+ MB


Let's inspect what information is stored in the playerType field

In [21]:
game_plays_players['playerType'].value_counts()

playerType
Shooter      1634308
Goalie        990290
Loser         894041
Winner        894041
Hittee        706098
Hitter        706098
PlayerID      485171
Blocker       431926
PenaltyOn     247733
Assist        245702
DrewBy        198271
Scorer        148992
ServedBy        3021
Unknown          912
Name: count, dtype: int64

It looks like the playerType is informative to the type of event happening in the play. Let's check for duplicate values.

In [23]:
duplicate_play_players = game_plays_players.groupby(['play_id', 'game_id', 'player_id'])['playerType'].count() > 1
duplicate_play_players.value_counts()

playerType
False    5138998
True     1223803
Name: count, dtype: int64

In [24]:
subset_columns = ['play_id', 'game_id', 'player_id']
game_plays_players = game_plays_players.drop_duplicates(subset=subset_columns, keep="first")

In [25]:
game_plays_players.shape

(6362801, 4)

In [26]:
game_plays_players.to_pickle(PICKLE_PATH / 'game_plays_players')

### Player Info 
This table contains biographical information about each player, including first and last name, nationality, birth city, position, birthday.

In [60]:
player_info = pd.read_csv(DATA_PATH / "player_info.csv")
player_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3925 entries, 0 to 3924
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   player_id           3925 non-null   int64  
 1   firstName           3925 non-null   object 
 2   lastName            3925 non-null   object 
 3   nationality         3917 non-null   object 
 4   birthCity           3920 non-null   object 
 5   primaryPosition     3925 non-null   object 
 6   birthDate           3925 non-null   object 
 7   birthStateProvince  2802 non-null   object 
 8   height              3922 non-null   object 
 9   height_cm           3922 non-null   float64
 10  weight              3922 non-null   float64
 11  shootsCatches       3908 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 368.1+ KB


There are no null values in fields of interest for our group. First, Last, birthday  (for age), and position are the only data points we plan to use.

In [61]:
duplicate_players = player_info.groupby(['player_id', 'firstName', 'lastName'])['nationality'].count() > 1
duplicate_players.value_counts()

nationality
False    3925
Name: count, dtype: int64

There are no duplicate players in the set.

### Added feature: allStarSeasons

We are going to add a feature for each player that contains the years in which they participated in the All Star Game. This can be used to differentiate top quality players in their position against others. Due to the variable skill level players can display throughout different phases of their career, we will use the years, rather than a binary allStar feature

First, we read in the all star data, containing season and player names for the all star team rosters

In [62]:
all_stars = pd.read_csv(DATA_PATH / 'all_star_data.csv')
all_stars.shape

(639, 2)

We need a column to merge on between the all star data and the player_info table, we can use full name (first + last)

In [63]:
player_info['fullName'] = player_info['firstName'] + ' ' + player_info['lastName']

We can merge the names, then inspect for any columns where we do not find a match. These will be due to non-matching names between the datasets.

In [64]:
merged = pd.merge(all_stars, player_info, left_on='Player', right_on='fullName', how='left')
merged[merged['player_id'].isna()]

,Season,Player,player_id,firstName,lastName,nationality,birthCity,primaryPosition,birthDate,birthStateProvince,height,height_cm,weight,shootsCatches,fullName


We iterated until there were no more missing rows. Most were conflicting spellings, use of nicknames, or use of special characters in names. We corrected these by editing the all star data csv to conform to the data in the player_info table

Now we can merge the data to create the allStarSeasons column, which will contain a list of the seasons in which the player was on an all star team roster.

In [65]:
player_seasons = merged.groupby('player_id')['Season'].apply(list).reset_index()
player_info = pd.merge(player_info, player_seasons, on='player_id', how='left').rename(columns={'Season': 'allStarSeasons'})
player_info

,player_id,firstName,lastName,nationality,birthCity,primaryPosition,birthDate,birthStateProvince,height,height_cm,weight,shootsCatches,fullName,allStarSeasons
0,8466148,Marian,Hossa,SVK,Stará Lubovna,RW,1979-01-12 00:00:00,NaN,"6' 1""",185.42,207.0,L,Marian Hossa,"[20002001, 20022003, 20062007, 20072008, 20112..."
1,8465058,Michal,Rozsival,CZE,Vlasim,D,1978-09-03 01:00:00,NaN,"6' 1""",185.42,210.0,R,Michal Rozsival,NaN
2,8476906,Shayne,Gostisbehere,USA,Pembroke Pines,D,1993-04-20 01:00:00,FL,"5' 11""",180.34,180.0,L,Shayne Gostisbehere,NaN
3,8466285,Brian,Campbell,CAN,Strathroy,D,1979-05-23 01:00:00,ON,"5' 10""",177.80,192.0,L,Brian Campbell,"[20062007, 20072008, 20082009, 20112012]"
4,8470607,Brent,Seabrook,CAN,Richmond,D,1985-04-20 01:00:00,BC,"6' 3""",190.50,220.0,R,Brent Seabrook,[20142015]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3920,8479374,Adam,Mascherin,CAN,Maple,LW,1998-06-06 01:00:00,ON,"5' 10""",177.80,205.0,L,Adam Mascherin,NaN
3921,8479403,Tyler,Parsons,USA,Mt. Clemas,G,1997-09-18 01:00:00,MI,"6' 1""",185.42,185.0,L,Tyler Parsons,NaN
3922,8480970,Marcus,Hogstrom,SWE,Sveg,D,1989-03-20 00:00:00,NaN,"6' 4""",193.04,200.0,L,Marcus Hogstrom,NaN
3923,8477574,Parker,Milner,USA,Pittsburgh,G,1990-09-06 01:00:00,PA,"6' 1""",185.42,185.0,L,Parker Milner,NaN


In [66]:
player_info.to_pickle(PICKLE_PATH / "player_info")